In [18]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [4]:
from sklearn.model_selection import train_test_split
import category_encoders

In [5]:
features = pd.read_csv("features_12.csv")
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [6]:
training = labels.set_index('person').join(features.set_index('person'))
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [7]:
predict_data = kaggle.set_index('person').join(features.set_index('person'))

In [8]:
encoder = category_encoders.TargetEncoder(smoothing=True)
train_data_enc = encoder.fit_transform(np.array(train_data),np.array(train_labels))
predict_data_enc = encoder.transform(np.array(predict_data))

In [43]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_enc,np.array(train_labels))

In [44]:
train = xgb.DMatrix(data=xtrain,label=ytrain)
test = xgb.DMatrix(data=xtest,label=ytest)

data = xgb.DMatrix(data=train_data_enc,label=np.array(train_labels))
pred = xgb.DMatrix(data=predict_data_enc)
params = {'colsample_bytree': 0.9,
 'gamma': 2,
 'learning_rate': 0.016238752458245277,
 'max_depth': 13,
 'min_child_weight': 5,
 'n_estimators': 100,
 'subsample': 0.7847190225361189,
 'eval_metric':'auc'}

In [45]:
bst = xgb.train(params,train,300,[(train,'train'),(test,'eval')])

[13:46:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 260 pruned nodes, max_depth=13
[0]	train-auc:0.845342	eval-auc:0.808495
[13:46:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 358 pruned nodes, max_depth=10
[1]	train-auc:0.851791	eval-auc:0.811142
[13:46:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 400 pruned nodes, max_depth=11
[2]	train-auc:0.866717	eval-auc:0.825035
[13:46:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 300 pruned nodes, max_depth=10
[3]	train-auc:0.866527	eval-auc:0.824472
[13:46:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 326 pruned nodes, max_depth=9
[4]	train-auc:0.868183	eval-auc:0.824703
[13:46:09] C:\Users\Admini

In [49]:
preds = bst.predict(test)

In [50]:
roc_auc_score(ytest,preds)

0.860695868522388

In [51]:
bst = xgb.train(params,data,200)

[13:51:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 382 pruned nodes, max_depth=13
[13:51:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 372 pruned nodes, max_depth=12
[13:51:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 414 pruned nodes, max_depth=13
[13:51:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 364 pruned nodes, max_depth=13
[13:51:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 362 pruned nodes, max_depth=11
[13:51:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 436 pruned nodes, max_depth=12
[13:51:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.c

In [52]:
xg_prediction = bst.predict(pred)

In [53]:
submit = kaggle.set_index('person')

In [54]:
submit['label'] = xg_prediction

In [55]:
submit.to_csv('submit1.csv')